# Projet ARF Inpainting

# BEROUKHIM Keyvan

## Préambule

In [55]:
import sklearn.linear_model
from sklearn.metrics import accuracy_score
import numpy as np

In [65]:
def load_usps(filename):
    with open(filename, "r") as f:
        f.readline()
        data = [[float(x) for x in l.split()] for l in f if len(l.split()) > 2]
    tmp = np.array(data)
    return tmp[:, 1:], tmp[:, 0].astype(int)

def keep_two(x, y, c0, c1):
    ind_kept = np.where((y==c0) | (y==c1))[0]
    return x[ind_kept], np.array([-1 if e==0 else 1 for e in y[ind_kept]])

In [66]:
c0 = 0
c1 = 1

In [67]:
train_x, train_y = keep_two(*load_usps("USPS_train.txt"), c0, c1)
test_x , test_y  = keep_two(*load_usps("USPS_test.txt" ), c0, c1)

In [71]:
models = [
    sklearn.linear_model.LinearRegression(),
    sklearn.linear_model.Ridge(alpha=.5),
    sklearn.linear_model.Lasso(alpha=.001)
]

In [69]:
for i, m in enumerate(models):
    print("régularisation", i)
    m.fit(train_x, train_y)
    print("précision train", accuracy_score(np.sign(m.predict(train_x)), train_y))
    print("précision test", accuracy_score(np.sign(m.predict(test_x)), test_y))
    print("nb poids non nuls", np.nonzero(m.coef_)[0].size)
    print()

régularisation 0
précision train 1.0
précision test 0.9839486356340289
nb poids non nuls 255

régularisation 1
précision train 1.0
précision test 0.9871589085072231
nb poids non nuls 254

régularisation 2
précision train 0.9995452478399273
précision test 0.9903691813804173
nb poids non nuls 97

